Use the following code to first test that your system can use the emailing library 'win32com.client' without issues. Remember to update 'mail.To' first.

In [6]:
import win32com.client as win32

# Initialize Outlook application
outlook = win32.Dispatch('outlook.application')

# Create a new email
mail = outlook.CreateItem(0)
mail.To = 'EXAMPLE@historicengland.org.uk'  # Replace with your HE email address
mail.Subject = 'Test Email'
mail.Body = 'This is a test email.'

# Send the email
mail.Send()
print("Test email sent.")


Test email sent.


If the email from the previous cell has been successfully sent and recieved, then the following code will be able to email all the relevant Heritage Indicator tables to contacts.

If the email from the previous cell has not been successfully sent and recieved, then there will need to be further investigation into your outlook/system settings, and is possibly due to the permissions given to outlook.

Remember to replace the filepaths before running the script.

In [ ]:
import pandas as pd
import os
import win32com.client as win32
from collections import defaultdict

# Define the file path containing 'HI Lookup Table and Metadata'
lookup_df = pd.read_excel(r'C:\filepath\HI Lookup Table and Metadata.xlsx')

# Define the folder path containing the Excel files
folder_path = r'C:\filepath\Updated HI Files Split Tables'

# Filter the DataFrame to include only rows where the status is 'Request'
requested_df = lookup_df[lookup_df['Data Source Type'] == 'Request']

# Group data by the recipient emails to identify unique recipient groups
email_columns = ['Email Source 1', 'Email Source 2', 'Email Source 3', 
                 'Email Source 4', 'Email Source 5', 'Email Source 6']

# Create a dictionary to hold the recipients and their respective files
recipient_dict = defaultdict(list)

# Create a dictionary to map file names to their descriptions
file_descriptions = {}

for _, row in requested_df.iterrows():
    # Gather all unique non-null emails for this row
    recipients = tuple(sorted(set(row[email_columns].dropna())))
    
    if not recipients:
        print(f"No valid email addresses for row: {row}")
        continue
    
    # Construct the file path, ensuring it has the correct .xlsx extension
    file_name = row['Sheet Name']
    if not file_name.endswith('.xlsx'):
        file_name += '.xlsx'  # Append .xlsx if missing
    
    file_path = os.path.join(folder_path, file_name)
    
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        continue
    
    # Map the file name (without the .xlsx extension) to its description
    file_descriptions[file_name] = row['Table Description - (Original Table Name)']
    
    # Append the file path to the recipients' list in the dictionary
    recipient_dict[recipients].append(file_path)

# Initialize Outlook application
outlook = win32.Dispatch('outlook.application')

# Function to generate the descriptive text for the attached files
def generate_description(files):
    description_text = ""
    for file in files:
        file_name = os.path.basename(file)  # Extract the file name
        file_name_without_extension = file_name.replace('.xlsx', '')  # Remove .xlsx
        if file_name in file_descriptions:
            description_text += f"{file_descriptions[file_name]} ({file_name_without_extension})\n"
        else:
            description_text += f"Description not found for {file_name_without_extension}\n"
    return description_text

# Function to send an email
def send_email(recipients, files_to_attach):
    try:
        mail = outlook.CreateItem(0)
        
        # Add recipients to the email
        for recipient in recipients:
            if recipient:  # Only add non-empty emails
                mail.Recipients.Add(recipient)

        # Generate the description text for the attached files
        descriptive_text = generate_description(files_to_attach)
        
        # Set subject and body
        mail.Subject = 'Heritage Counts Data Request 2024'
        mail.Body = f"""Hello,

I am reaching out to request the data for this year’s Heritage Counts publication. We would greatly appreciate it if you could provide the latest data, as you have kindly done in the past.

Attached, you will find the spreadsheets ready for your input. The spreadsheets are named numerically, with their descriptive titles listed below for your reference:
{descriptive_text}

If you have any questions or encounter any issues with the spreadsheets, such as discrepancies in past annual data, please do not hesitate to contact me.

We would be grateful if you could return the required 2024 data by Friday, 4th October. If this deadline poses any challenges, or if you are no longer the appropriate contact for this dataset, please let me know so we can update our records accordingly.

Thank you for your cooperation and support.
        
Best regards,
The Heritage Counts Team"""
        
        # Attach files
        for file in files_to_attach:
            if os.path.exists(file):  # Check if the file exists
                mail.Attachments.Add(file)
            else:
                print(f"Warning: File {file} not found. It will not be attached.")
        
        # Send the email
        mail.Send()
        print(f"Email sent to {', '.join(recipients)} with {len(files_to_attach)} attachment(s).")
    
    except Exception as e:
        print(f"Failed to send email to {', '.join(recipients)}. Error: {e}")

# Iterate through the recipient dictionary and send emails
for recipients, files in recipient_dict.items():
    if recipients:  # Ensure there are recipients to send the email to
        print(f"Preparing to send email to: {', '.join(recipients)} with files: {files}")
        send_email(recipients, files)
    else:
        print("No valid email addresses found for one or more entries.")

print("All emails have been processed.")
